# Neural Collaborative Filtering (NCF)
Use more deep learning and collaborative filtering for recommenders systems.

In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure # use pure ratings only
from libreco.algorithms import NCF
from libreco.evaluation import evaluate

In [2]:
ratings_path = "../data/ml-latest-small/ratings.csv"
movies_path = "../data/ml-latest-small/movies.csv"

ratings = pd.read_csv(ratings_path)
movies = pd.read_csv(movies_path)

ratings.columns = ["user", "item", "label", "time"]
movies.columns = ["movieID", "Title", "Genres"]

In [3]:
# random split
training_set, evaluation_set, testing_set = random_split(ratings, multi_ratios=[0.8, 0.1, 0.1])
# convert into LibRecommender data format
training_set, data_info = DatasetPure.build_trainset(training_set)
evaluation_set = DatasetPure.build_evalset(evaluation_set)
testing_set = DatasetPure.build_testset(testing_set)

In [ ]:
# define model instance
ncf = NCF(task="rating", data_info=data_info)

# train
ncf.fit(
    training_set,
    neg_sampling=False, # use only explicit-ratings
    verbose=2,
    eval_data=evaluation_set,
    metrics=["loss"]
)

In [ ]:
# evaluate
evaluate(
    model=ncf,
    data=testing_set,
    neg_sampling=False,
    metrics=["loss"]
)

In [ ]:
# print a sample user 1 rated to know a little bit more about tastes
for idx, row in ratings[ratings.user==1].sample(10):
    rating = row['label']
    movie = movies[movies.movieID==row['item']]
    print(rating, movie['Title'])

In [ ]:
# predict rating of user 1 of movie 1 (toystory)
print(ncf.predict(user=1, item=1))
# now recommend 10 items for user 1
recs = ncf.recommend_user(user=1, n_rec=10)
for rec in recs[1]:
    print(movies[movies.movieID==rec].Title)